#Pretrained models


## Keras

In [3]:
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

In [ ]:
import keras
dir(keras.applications)

In [5]:
vgg_model = VGG16(weights='imagenet')

553467096/553467096 [==============================] - 5s 0us/step


In [21]:
path_img=['1.jpg', '2.jpg']
for i in range(len(path_img)):
  my_img = load_img(path_img[i], target_size=[224, 224, 3])
  my_img = img_to_array(my_img)
  my_img = np.expand_dims(my_img, axis=0)
  my_img = preprocess_input(my_img)
  pred = vgg_model.predict(my_img)
  top_acc = 5
  for j in range(top_acc):
    print(decode_predictions(pred, top=top_acc)[0][j][1])
  print('--------------------')

1/1 [==============================] - 0s 18ms/step
bulbul
hummingbird
junco
jay
magpie
--------------------
1/1 [==============================] - 0s 28ms/step
jay
partridge
ruffed_grouse
quail
ruddy_turnstone
--------------------


## PyTorch

In [33]:
import torch
from torchvision.models import vgg16
from torchvision import transforms
from PIL import Image

In [ ]:
import torchvision
dir(torchvision.models)

In [ ]:
torch_vgg_model = vgg16(pretrained=True)
# feedforward
torch_vgg_model.eval()

In [35]:
preprocess = transforms.Compose([transforms.Resize(256),
                    transforms.CenterCrop(224),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                         std=[0.229, 0.224, 0.225])
                    ])
img = Image.open(path_img[1])
img = preprocess(img)
batch_img = torch.unsqueeze(img, 0)


with torch.no_grad():
  pred = torch_vgg_model(batch_img)


_, indices = torch.sort(pred, descending=True)
print(indices[0][:5])

tensor([ 92, 133, 139,  86, 138])


In [38]:
with open("imagenet_class_name.txt") as f:
  classes = [line.strip() for line in f.readlines()]

In [40]:
_, index = torch.max(pred, 1)
percentage = torch.nn.functional.softmax(pred, dim=1)[0] * 100
print(classes[index[0]], percentage[index[0]].item())

91,coucal 26.01412582397461


## ResNet

In [ ]:
from torchvision.models import resnet50, ResNet50_Weights

In [ ]:
# Old weights with accuracy 76.130%
RN50v1 = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)

# New weights with accuracy 80.858%
RN50v2 = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
# Strings are also supported
# resnet50(weights="IMAGENET1K_V2")

# Best available weights (currently alias for IMAGENET1K_V2)
# Note that these weights may change across versions
RN50vd = resnet50(weights=ResNet50_Weights.DEFAULT)

# No weights - random initialization
RN50None = resnet50(weights=None)



In [43]:
# from torchvision.models.quantization.inception import Inception_V3_QuantizedWeights
from torchvision.io import read_image
from torchvision.models.quantization import resnet50, ResNet50_QuantizedWeights

img = read_image(path_img[1])

# Step 1: Initialize model with the best available weights
weights = ResNet50_QuantizedWeights.DEFAULT
model = resnet50(weights=weights, quantize=True)
# feed-forward
model.eval()

# Step 2: Initialize the inference transforms
preprocess = weights.transforms()

# Step 3: Apply inference preprocessing transforms
batch = preprocess(img).unsqueeze(0)

# Step 4: Use the model and print the predicted category
prediction = model(batch).squeeze(0).softmax(0)
class_id = prediction.argmax().item()
score = prediction[class_id].item()
category_name = weights.meta["categories"][class_id]
print(f"{category_name}: {100 * score}%")

jay: 25.964227318763733%
